In [1]:
import pandas as pd
import numpy as np

In [ ]:
#funciones

def map_category(category_text):
    """
    Aplica el mapeo para asignar una Categoría de Nivel 1.
    """
    if pd.isna(category_text):
        return np.nan # Dejar el NaN original para su posterior imputación
    
    text = str(category_text).lower()
    
    for keyword in sorted_keywords:
        if keyword in text:
            # Devuelve la primera coincidencia (la más larga/específica)
            return category_mapping[keyword]
            
    # Si no se encuentra ninguna palabra clave
    return "Otros / Sin Coincidencia"

#### Lectura csv original

In [ ]:
df = pd.read_csv("../Data/openfoodfacts_export.csv", sep="\t", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Columns: 458 entries, code to data_sources
dtypes: float64(159), int64(1), object(298)
memory usage: 13.6+ MB


In [ ]:
df = df[lista_columnas]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3124 non-null   object 
 2   serving_size                 1015 non-null   object 
 3   brands                       3646 non-null   object 
 4   categories                   3588 non-null   object 
 5   stores                       2221 non-null   object 
 6   nutrition_data_per           3878 non-null   object 
 7   nutrition_data_prepared_per  3878 non-null   object 
 8   energy-kcal_value            2729 non-null   float64
 9   energy-kcal_unit             2732 non-null   object 
 10  energy-kcal_prepared_value   14 non-null     float64
 11  energy-kcal_prepared_unit    15 non-null     object 
 12  fat_value                    2923 non-null   float64
 13  fat_unit          

#### Eliminar columnas que tengan menos del 30% de sus valores 

In [ ]:
df = df.dropna(axis=1, thresh = int(0.7*len(df)))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3124 non-null   object 
 2   brands                       3646 non-null   object 
 3   categories                   3588 non-null   object 
 4   nutrition_data_per           3878 non-null   object 
 5   nutrition_data_prepared_per  3878 non-null   object 
 6   energy-kcal_value            2729 non-null   float64
 7   energy-kcal_unit             2732 non-null   object 
 8   fat_value                    2923 non-null   float64
 9   fat_unit                     2925 non-null   object 
 10  saturated-fat_value          2842 non-null   float64
 11  saturated-fat_unit           2845 non-null   object 
 12  carbohydrates_value          2917 non-null   float64
 13  carbohydrates_unit

#### Eliminar productos de los que no tenemos los nombres. No nos sirven

In [ ]:
df = df.dropna(subset="product_name_es")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

#### Seleccionamos las columnas que nos convienen para el estudio

### Mapeo de categorias

#### Crear diccionario para mapear las categorias de alimentos y trabajar por categorias 

In [ ]:
df["categories"].unique()

array(['Condimentos, Salsas, en:Meal sauces, Salsas de tomate, Salsas de tomate frito, en:Groceries',
       'Lácteos, Comidas fermentadas, Productos fermentados de la leche, Postres, Postres lácteos, en:Fermented dairy desserts, en:Kefir yogurts',
       'Alimentos y bebidas de origen vegetal, Alimentos de origen vegetal, Desayunos, Cereales y patatas, Cereales y derivados, Cereales para el desayuno, Copos, Copos de cereales, en:Extruded cereals, en:Extruded flakes, Copos de maíz tostado',
       ..., 'Aditivos alimentarios, Colorantes alimentarios',
       'Comidas preparadas, Ecológica',
       'Artesanal, Salchichon-extra-taco-biobardales-350gr'],
      shape=(1706,), dtype=object)

#### Usar el diccionario para mapear

In [ ]:
columna_cat = "categories"
columna_prod = "product_name_es" # Columna de rescate

try:
    # --- PRIMER PASO: Mapeo de la columna 'categories' (Prioridad Alta) ---
    print(f"1. Aplicando mapeo a la columna principal '{columna_cat}'...")
    df['Categoria'] = df[columna_cat].apply(map_category)
    
    # --- SEGUNDO PASO: Rescate de NaN y 'Otros' usando 'product_name_es' ---
    
    # 2.1 Identificar filas que necesitan rescate (NaN o 'Otros / Sin Coincidencia')
    nan_o_otros_rows = df[df['Categoria'].isna() | (df['Categoria'] == "Otros / Sin Coincidencia")]
    
    print(f"2. Rescatando {len(nan_o_otros_rows)} filas usando '{columna_prod}'...")

    # 2.2 Mapear solo la columna de producto para estas filas
    imputacion_rescate = nan_o_otros_rows[columna_prod].apply(map_category)

    # 2.3 Rellenar los valores en el DataFrame principal
    # Usamos .combine_first() para rellenar los valores NaN/Otros con los nuevos mapeos de rescate
    df['Categoria'] = df['Categoria'].combine_first(imputacion_rescate)
    
    # 2.4 Reemplazar los marcadores temporales "Otros / Sin Coincidencia" con los valores de rescate
    # Esto es vital para que los productos mapeados por product_name_es se clasifiquen.
    # Usamos .loc para actualizar solo donde Categoria es el marcador temporal
    df.loc[df['Categoria'] == "Otros / Sin Coincidencia", 'Categoria'] = imputacion_rescate.loc[df['Categoria'] == "Otros / Sin Coincidencia"]


    # --- TERCER PASO: Imputación Final de NaN y Etiquetado Final ---
    
    # Contamos los NaN que quedan (eran NaN en categories y no se mapearon por product_name_es)
    remaining_nan_count = df['Categoria'].isna().sum()
    
    # 3.1 Rellenar los NaN restantes con "Categoría Desconocida"
    df['Categoria'].fillna("Categoría Desconocida", inplace=True)
    
    # 3.2 Etiquetar los productos restantes como "Otros" (los que no se pudieron mapear NUNCA)
    df['Categoria'] = df['Categoria'].replace('Otros / Sin Coincidencia', 'Otros (No Clasificado)')
    
    print(f"3. Imputación final: {remaining_nan_count} NaN a 'Categoría Desconocida'.")
    
    # --- VERIFICACIÓN Y RESULTADOS ---
    
    print("\n--- Recuento de Frecuencia de Categoria Final ---")
    print(df['Categoria'].value_counts())

except KeyError as e:
    print(f"Error: No se encontró la columna {e}. Verifica los nombres de las columnas ('categories', 'product_name_es').")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

1. Aplicando mapeo a la columna principal 'categories'...
2. Rescatando 396 filas usando 'product_name_es'...
3. Imputación final: 0 NaN a 'Categoría Desconocida'.

--- Recuento de Frecuencia de Categoria Final ---
Categoria
Frutas, Verduras y Legumbres               880
Lácteos y Derivados                        653
Snacks, Dulces y Postres                   414
Carne, Aves y Productos Cárnicos           365
Cereales, Panadería y Pasta                316
Salsas, Condimentos y Aditivos             292
Bebidas Alcohólicas                        176
Comidas Preparadas y Envasadas             164
Bebidas, Agua e Infusiones                 154
Aceites, Grasas y Productos Similares      153
Pescados, Mariscos y Derivados             112
Suplementos y Artículos No Alimentarios     32
Name: count, dtype: int64


In [ ]:
df_no_clasificados = df[df['Categoria'] == 'Otros (No Clasificado)']

# 2. Mostrar las columnas de texto clave para la revisión
columnas_revision = 'product_name_es'

print("--- Revisión de Productos en 'Otros (No Clasificado)' ---")
print(f"Número total de filas a revisar: {len(df_no_clasificados)}")
df_no_clasificados[columnas_revision].tolist()

--- Revisión de Productos en 'Otros (No Clasificado)' ---
Número total de filas a revisar: 0


[]

In [ ]:
# Guardar el resultado final
df.to_csv("df_util_mapeado.csv", index=False)
print("\nProceso de imputación completado. El archivo guardado es: df_util_mapeado.csv")


Proceso de imputación completado. El archivo guardado es: df_util_mapeado.csv


#### Comprobar nulos

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

#### Rellenar NaN de las categorias necesarias

Vamos a rellenar con las medianas por categoria los valores nulos que tenemos en cada categoria

Primero rellenaremos con 0 los valores de Agua ya que no tiene calorias ni azucares ni grasas etc

In [ ]:
columnas_a_rellenar = ['energy-kcal_value', 'fat_value', 'sugars_value', 'salt_value']

#Imputar 0 a los productos de agua
mascara_agua = (df['Categoria'] == 'Bebidas, Agua e Infusiones') & (df['product_name_es'].str.lower().str.contains('agua', na=False))

df.loc[mascara_agua, columnas_a_rellenar] = 0



In [ ]:
#Rellenar los NaN restantes con la mediana por categoría usando .transform()
for col in columnas_a_rellenar:
    medianas_por_grupo = df.groupby('Categoria')[col].transform('median') #calcula medianas por grupo
    df[col].fillna(medianas_por_grupo, inplace=True) # Rellena los NaN de la columna original con los valores de la Serie calculada

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            3711 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    3711 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

Vamos a comprobar y a eliminar ahora los registros que no tienen nutriscore y que no tienen Nova

In [ ]:
nutriscore_validos = ['a', 'b', 'c', 'd', 'e']

# 1. Filtramos para obtener solo los registros NO válidos (incluye NaN)
df_invalidos = df[~df['off:nutriscore_grade'].isin(nutriscore_validos)]

# 2. Obtenemos la lista de valores únicos de estos registros no válidos
valores_no_validos = df_invalidos['off:nutriscore_grade'].unique()
print (f"Hay {len(df_invalidos)} en el conjunto")

print("--- Valores Únicos en off:nutriscore_grade a Eliminar ---")
print("Los siguientes valores NO son 'a', 'b', 'c', 'd' o 'e':")
print(valores_no_validos)

Hay 1071 en el conjunto
--- Valores Únicos en off:nutriscore_grade a Eliminar ---
Los siguientes valores NO son 'a', 'b', 'c', 'd' o 'e':
['unknown' 'not-applicable' nan]


Como son muchos, convertimos los valores no validos a Nan y los cambiamos por los mas frecuentes en su grupo

In [ ]:
df['nutriscore_limpio'] = df['off:nutriscore_grade'].copy() #nueva columna con los valores 

es_invalido = ~df['nutriscore_limpio'].isin(nutriscore_validos)

df.loc[es_invalido, 'nutriscore_limpio'] = np.nan

In [ ]:
moda_por_categoria = df.groupby('Categoria')['nutriscore_limpio'].apply(
    lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
)
moda_por_categoria



Categoria
Aceites, Grasas y Productos Similares        b
Bebidas Alcohólicas                        NaN
Bebidas, Agua e Infusiones                   a
Carne, Aves y Productos Cárnicos             e
Cereales, Panadería y Pasta                  d
Comidas Preparadas y Envasadas               a
Frutas, Verduras y Legumbres                 a
Lácteos y Derivados                          b
Pescados, Mariscos y Derivados               d
Salsas, Condimentos y Aditivos               a
Snacks, Dulces y Postres                     e
Suplementos y Artículos No Alimentarios      a
Name: nutriscore_limpio, dtype: object

In [ ]:
df['off:nutriscore_grade'] = df['nutriscore_limpio'].fillna(df['Categoria'].map(moda_por_categoria))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            3711 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    3711 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

In [ ]:
registros_nan_nutriscore = df[df['off:nutriscore_grade'].isna()]
conteo_por_categoria_nan = registros_nan_nutriscore['Categoria'].value_counts()
conteo_por_categoria_nan

Categoria
Bebidas Alcohólicas    176
Name: count, dtype: int64

Como las que no tienen nutriscore despues de rellenar son bebidas alcoholicas que no nos interesan en el estudio, las eliminamos

In [ ]:
df.dropna(subset=['off:nutriscore_grade'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3535 entries, 0 to 3877
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3535 non-null   object 
 1   quantity                     2892 non-null   object 
 2   brands                       3350 non-null   object 
 3   categories                   3286 non-null   object 
 4   nutrition_data_per           3535 non-null   object 
 5   nutrition_data_prepared_per  3535 non-null   object 
 6   energy-kcal_value            3535 non-null   float64
 7   energy-kcal_unit             2607 non-null   object 
 8   fat_value                    3535 non-null   float64
 9   fat_unit                     2794 non-null   object 
 10  saturated-fat_value          2715 non-null   float64
 11  saturated-fat_unit           2718 non-null   object 
 12  carbohydrates_value          2787 non-null   float64
 13  carbohydrates_unit     

In [ ]:
df.dropna(subset=['off:nova_groups_tags'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3534 entries, 0 to 3877
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3534 non-null   object 
 1   quantity                     2891 non-null   object 
 2   brands                       3349 non-null   object 
 3   categories                   3285 non-null   object 
 4   nutrition_data_per           3534 non-null   object 
 5   nutrition_data_prepared_per  3534 non-null   object 
 6   energy-kcal_value            3534 non-null   float64
 7   energy-kcal_unit             2607 non-null   object 
 8   fat_value                    3534 non-null   float64
 9   fat_unit                     2794 non-null   object 
 10  saturated-fat_value          2715 non-null   float64
 11  saturated-fat_unit           2718 non-null   object 
 12  carbohydrates_value          2787 non-null   float64
 13  carbohydrates_unit     

In [ ]:
df.drop(columns=['nutriscore_limpio'], inplace=True) #ya no la necesitamos

In [ ]:
df.to_csv("df_util_sin_bebidas.csv")

Ahora limpiamos el csv cesta_spa.csv

In [ ]:
df_cesta = pd.read_csv("../Data/cesta_spa.csv", sep="\t", encoding="latin-1")
df_cesta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Códigos de gasto con cantidad física  60 non-null     object
 1   Cantidades medias consumidas          60 non-null     object
 2   Periodo                               60 non-null     int64 
 3   Total                                 60 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.0+ KB


In [ ]:

#Limpieza de Consumo_Total: Reemplazar coma por punto y convertir a float

df_cesta['Total'] = df_cesta['Total'].astype(str).str.replace(',', '.', regex=False).astype(float)

# 2. Crear la columna 'Producto' (solo la descripción)
#    Se divide la columna original por el primer espacio y se toma el segundo elemento
df_cesta['Producto'] = df_cesta['Códigos de gasto con cantidad física'].str.split(' ', n=1).str[1].str.strip()

print("Columna 'Producto' creada y Columna 'Total' convertida a numérica:")
print(df_cesta[['Producto', 'Total']].head())
df_cesta.info()

Columna 'Producto' creada y Columna 'Total' convertida a numérica:
                                            Producto  Total
0                                       Arroz (kilo)    5.8
1                                         Pan (kilo)   44.8
2                Otros productos de panadería (kilo)   12.0
3                         Pastas alimenticias (kilo)    5.1
4  Carne de bovino fresca, refrigerada o congelad...    4.7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Códigos de gasto con cantidad física  60 non-null     object 
 1   Cantidades medias consumidas          60 non-null     object 
 2   Periodo                               60 non-null     int64  
 3   Total                                 60 non-null     float64
 4   Producto                              60 non-null     object 
dt

In [ ]:
df_cesta["Producto"].unique().tolist()

['Arroz (kilo)',
 'Pan (kilo)',
 'Otros productos de panadería (kilo)',
 'Pastas alimenticias (kilo)',
 'Carne de bovino fresca, refrigerada o congelada (kilo)',
 'Carne de porcino fresca, refrigerada o congelada (kilo)',
 'Carne de ovino y caprino fresca, refrigerada o congelada (kilo)',
 'Carne de ave fresca, refrigerada o congelada (kilo)',
 'Charcutería y carne seca, salada o ahumada (kilo)',
 'Carnes preparadas y otros productos conteniendo carne (kilo)',
 'Otras carnes comestibles (frescas y congeladas) (kilo)',
 'Despojos, menudillos y casquería (kilo)',
 'Pescados frescos o refrigerados (kilo)',
 'Pescados congelados (kilo)',
 'Crustáceos y moluscos frescos, refrigerados o congelados (kilo)',
 'Pescados y mariscos secos, ahumados o salados (kilo)',
 'Otros pescados y mariscos procesados o conservados y preparados de pescados y mariscos (kilo)',
 'Leche entera (litro)',
 'Leche semidescremada y descremada (litro)',
 'Leche conservada (kilo)',
 'Yogures y leches fermentadas (kilo

In [ ]:

# Aplicar la función de mapeo a la columna 'Producto'
df_cesta['Categoria_Final'] = df_cesta['Producto'].apply(map_category)

# Agrupar el consumo total ('Total') por la nueva columna 'Categoria_Final' y sumar
consumo_agregado = df_cesta.groupby('Categoria_Final')['Total'].sum().sort_values(ascending=False)

print("\nConsumo Medio Total (en kilos/litros) por Persona y por Categoría (Modelo Combinado):")
print("--------------------------------------------------------------------------------------")
print(consumo_agregado)
print("--------------------------------------------------------------------------------------")


Consumo Medio Total (en kilos/litros) por Persona y por Categoría (Modelo Combinado):
--------------------------------------------------------------------------------------
Categoria_Final
Lácteos y Derivados                      242.0
Frutas, Verduras y Legumbres             174.8
Bebidas, Agua e Infusiones               152.4
Cereales, Panadería y Pasta               67.7
Carne, Aves y Productos Cárnicos          52.0
Pescados, Mariscos y Derivados            21.3
Aceites, Grasas y Productos Similares     14.4
Snacks, Dulces y Postres                  10.7
Name: Total, dtype: float64
--------------------------------------------------------------------------------------


In [ ]:
df_cesta

,Códigos de gasto con cantidad física,Cantidades medias consumidas,Periodo,Total,Producto,Categoria_Final
0,01111 Arroz (kilo),Cantidad consumida media por persona,2015,5.8,Arroz (kilo),"Cereales, Panadería y Pasta"
1,01112 Pan (kilo),Cantidad consumida media por persona,2015,44.8,Pan (kilo),"Cereales, Panadería y Pasta"
2,01113 Otros productos de panadería (kilo),Cantidad consumida media por persona,2015,12.0,Otros productos de panadería (kilo),"Cereales, Panadería y Pasta"
3,01114 Pastas alimenticias (kilo),Cantidad consumida media por persona,2015,5.1,Pastas alimenticias (kilo),"Cereales, Panadería y Pasta"
4,"01121 Carne de bovino fresca, refrigerada o co...",Cantidad consumida media por persona,2015,4.7,"Carne de bovino fresca, refrigerada o congelad...","Carne, Aves y Productos Cárnicos"
5,"01122 Carne de porcino fresca, refrigerada o c...",Cantidad consumida media por persona,2015,7.6,"Carne de porcino fresca, refrigerada o congela...","Carne, Aves y Productos Cárnicos"
6,"01123 Carne de ovino y caprino fresca, refrige...",Cantidad consumida media por persona,2015,1.2,"Carne de ovino y caprino fresca, refrigerada o...","Carne, Aves y Productos Cárnicos"
7,"01124 Carne de ave fresca, refrigerada o conge...",Cantidad consumida media por persona,2015,13.8,"Carne de ave fresca, refrigerada o congelada (...","Carne, Aves y Productos Cárnicos"
8,"01125 Charcutería y carne seca, salada o ahuma...",Cantidad consumida media por persona,2015,16.5,"Charcutería y carne seca, salada o ahumada (kilo)","Carne, Aves y Productos Cárnicos"
9,01126 Carnes preparadas y otros productos cont...,Cantidad consumida media por persona,2015,6.0,Carnes preparadas y otros productos conteniend...,"Carne, Aves y Productos Cárnicos"


In [ ]:
df_cesta["Producto"].unique().tolist()

['Arroz (kilo)',
 'Pan (kilo)',
 'Otros productos de panadería (kilo)',
 'Pastas alimenticias (kilo)',
 'Carne de bovino fresca, refrigerada o congelada (kilo)',
 'Carne de porcino fresca, refrigerada o congelada (kilo)',
 'Carne de ovino y caprino fresca, refrigerada o congelada (kilo)',
 'Carne de ave fresca, refrigerada o congelada (kilo)',
 'Charcutería y carne seca, salada o ahumada (kilo)',
 'Carnes preparadas y otros productos conteniendo carne (kilo)',
 'Otras carnes comestibles (frescas y congeladas) (kilo)',
 'Despojos, menudillos y casquería (kilo)',
 'Pescados frescos o refrigerados (kilo)',
 'Pescados congelados (kilo)',
 'Crustáceos y moluscos frescos, refrigerados o congelados (kilo)',
 'Pescados y mariscos secos, ahumados o salados (kilo)',
 'Otros pescados y mariscos procesados o conservados y preparados de pescados y mariscos (kilo)',
 'Leche entera (litro)',
 'Leche semidescremada y descremada (litro)',
 'Leche conservada (kilo)',
 'Yogures y leches fermentadas (kilo

In [ ]:
nombre_diccionario = {
    'Arroz (kilo)': 'arroz',
    'Pan (kilo)': 'pan',
    'Otros productos de panadería (kilo)': 'panadería',
    'Pastas alimenticias (kilo)': 'pasta',
    'Carne de bovino fresca, refrigerada o congelada (kilo)': 'carne vaca',
    'Carne de porcino fresca, refrigerada o congelada (kilo)': 'carne cerdo',
    'Carne de ovino y caprino fresca, refrigerada o congelada (kilo)': 'carne cordero',
    'Carne de ave fresca, refrigerada o congelada (kilo)': 'carne ave',
    'Charcutería y carne seca, salada o ahumada (kilo)': 'charcutería',
    'Carnes preparadas y otros productos conteniendo carne (kilo)': 'carnes preparadas',
    'Otras carnes comestibles (frescas y congeladas) (kilo)': 'otras carnes',
    'Despojos, menudillos y casquería (kilo)': 'casqueria',
    'Pescados frescos o refrigerados (kilo)': 'pescado fresco',
    'Pescados congelados (kilo)': 'pescado congelado',
    'Crustáceos y moluscos frescos, refrigerados o congelados (kilo)': 'crustáceos/moluscos',
    'Pescados y mariscos secos, ahumados o salados (kilo)': 'pescado/marisco seco',
    'Otros pescados y mariscos procesados o conservados y preparados de pescados y mariscos (kilo)': 'pescado/marisco procesado',
    'Leche entera (litro)': 'leche entera',
    'Leche semidescremada y descremada (litro)': 'leche descremada',
    'Leche conservada (kilo)': 'leche conservada',
    'Yogures y leches fermentadas (kilo)': 'yogur',
    'Queso y requesón (kilo)': 'queso',
    'Otros productos a base de leche (kilo)': 'otros lácteos',
    'Huevos (unidad)': 'huevos',
    'Mantequilla (kilo)': 'mantequilla',
    'Margarina y otras grasas vegetales (kilo)': 'grasas vegetales',
    'Aceite de oliva (litro)': 'aceite oliva',
    'Otros aceites comestibles (litro)': 'otros aceites',
    'Otras grasas animales (kilo)': 'grasas animales',
    'Cítricos (frescos, refrigerados o congelados) (kilo)': 'cítricos',
    'Plátanos (frescos, refrigerados o congelados) (kilo)': 'plátanos',
    'Manzanas (frescas, refrigeradas o congeladas) (kilo)': 'manzanas',
    'Peras (frescas, refrigeradas o congeladas) (kilo)': 'peras',
    'Frutas con hueso (frescas, refrigeradas o congeladas) (kilo)': 'frutas con hueso',
    'Aceitunas (kilo)': 'aceitunas',
    'Bayas (frescas, refrigeradas o congeladas) (kilo)': 'bayas',
    'Otras frutas (frescas, refrigeradas o congeladas) (kilo)': 'otras frutas',
    'Frutos secos (kilo)': 'frutos secos',
    'Frutas preparadas y en conserva (kilo)': 'frutas preparadas',
    'Hortalizas de hoja o de tallo (frescas o refrigeradas) (kilo)': 'hortalizas hoja/tallo',
    'Coles (frescas o refrigeradas) (kilo)': 'coles',
    'Hortalizas cultivadas por su fruto (frescas o refrigeradas) (kilo)': 'hortalizas fruto',
    'Hortalizas con raíz o bulbo y setas (frescas o refrigeradas) (kilo)': 'hortalizas raíz/bulbo',
    'Legumbres y hortalizas secas (kilo)': 'legumbres secas',
    'Verduras congeladas (kilo)': 'verduras congeladas',
    'Legumbres y hortalizas en conserva o preparadas y otros productos a base de legumbres y hortalizas (kilo)': 'legumbres/verduras preparadas',
    'Patatas (kilo)': 'patatas',
    'Productos derivados de las patatas, mandioca y otros tubérculos (kilo)': 'derivados de patata',
    'Azúcar (kilo)': 'azúcar',
    'Confitura, mermelada y miel (kilo)': 'dulces',
    'Chocolate en barra o en tableta (kilo)': 'chocolate',
    'Helados (kilo)': 'helados',
    'Otros productos a base de azúcar (kilo)': 'otros dulces',
    'Café (kilo)': 'café',
    'Té e infusiones (kilo)': 'té/infusiones',
    'Cacao (kilo)': 'cacao',
    'Agua mineral (litro)': 'agua mineral',
    'Bebidas refrescantes con o sin gas (litro)': 'refrescos',
    'Zumos de frutas (litro)': 'zumos frutas',
    'Zumos de vegetales (litro)': 'zumos vegetales'
}

# Aplicamos el diccionario directamente sobre df_cesta
df_cesta['Producto_limpio'] = df_cesta['Producto'].map(nombre_diccionario)

# Si algún producto no está en el diccionario, se mantiene el nombre original
df_cesta['Producto_limpio'] = df_cesta['Producto_limpio'].fillna(df_cesta['Producto'])

df_cesta[['Producto', 'Producto_limpio']].head(10)
df_cesta.to_csv("../Data/cesta_spa_limpio.csv")


In [2]:
df_2 = pd.read_excel('../data/columnas_dataset.xlsx')

In [3]:
lista_columnas = df_2.columns.tolist()
lista_columnas

['product_name_es',
 'quantity',
 'serving_size',
 'brands',
 'categories',
 'stores',
 'nutrition_data_per',
 'nutrition_data_prepared_per',
 'energy-kcal_value',
 'energy-kcal_unit',
 'energy-kcal_prepared_value',
 'energy-kcal_prepared_unit',
 'fat_value',
 'fat_unit',
 'saturated-fat_value',
 'saturated-fat_unit',
 'carbohydrates_value',
 'carbohydrates_unit',
 'sugars_value',
 'sugars_unit',
 'fiber_value',
 'fiber_unit',
 'proteins_value',
 'proteins_unit',
 'salt_value',
 'salt_unit',
 'sodium_value',
 'sodium_unit',
 'energy_value',
 'energy_unit',
 'off:food_groups',
 'off:food_groups_tags',
 'off:nova_groups',
 'off:nova_groups_tags',
 'off:nutriscore_grade']

lectura el csv original

In [5]:
df = pd.read_csv("../Data/openfoodfacts_export.csv", sep="\t", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Columns: 458 entries, code to data_sources
dtypes: float64(159), int64(1), object(298)
memory usage: 13.6+ MB


In [6]:
df = df[lista_columnas]

In [7]:
df

,product_name_es,quantity,serving_size,brands,categories,stores,nutrition_data_per,nutrition_data_prepared_per,energy-kcal_value,energy-kcal_unit,...,salt_unit,sodium_value,sodium_unit,energy_value,energy_unit,off:food_groups,off:food_groups_tags,off:nova_groups,off:nova_groups_tags,off:nutriscore_grade
0,Tomate frito,400 g,1 serving (100 g),Hacendado,"Condimentos, Salsas, en:Meal sauces, Salsas de...",Mercadona,100g,serving,77.0,kcal,...,g,0.393,g,77.0,kcal,en:dressings-and-sauces,"en:fats-and-sauces,en:dressings-and-sauces",4.0,en:4-ultra-processed-food-and-drink-products,c
1,Kéfir natural sabor suave,500 g,1.0 vaso (250.0 g),Hacendado,"Lácteos, Comidas fermentadas, Productos fermen...",Mercadona,100g,100g,74.0,kcal,...,mg,0.032,mg,308.0,kJ,en:dairy-desserts,"en:milk-and-dairy-products,en:dairy-desserts",1.0,en:1-unprocessed-or-minimally-processed-foods,b
2,Corn Flakes 0% azúcares añadidos,30,NaN,Hacendado,"Alimentos y bebidas de origen vegetal, Aliment...",Mercadona,100g,100g,373.0,kcal,...,g,0.400,g,1583.0,kJ,en:breakfast-cereals,"en:cereals-and-potatoes,en:breakfast-cereals",3.0,en:3-processed-foods,c
3,Galletas de trigo espelta 0% azúcares añadidos,200 g,10 g,Hacendado,"Botanas, Snacks dulces, Galletas y pasteles, e...",Mercadona,100g,100g,422.0,kcal,...,g,0.354,g,422.0,kcal,en:biscuits-and-cakes,"en:sugary-snacks,en:biscuits-and-cakes",4.0,en:4-ultra-processed-food-and-drink-products,b
4,Tomate frito,630 g (3 x 210 g),210g,Hacendado,"Condimentos, Salsas, en:Meal sauces, Salsas de...",Mercadona,100g,100g,81.0,kcal,...,mg,0.000,mg,329.0,kJ,en:dressings-and-sauces,"en:fats-and-sauces,en:dressings-and-sauces",3.0,en:3-processed-foods,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3873,Potón en tinta,Peso neto: 240 g (80 g x 3),80 g,"Aldi, Sal de plata","Productos del mar, en:Fishes and their product...",Aldi,100g,100g,212.0,kcal,...,g,0.280,g,878.0,kJ,en:fish-and-seafood,"en:fish-meat-eggs,en:fish-and-seafood",4.0,en:4-ultra-processed-food-and-drink-products,b
3874,Colorante alimentario,85g,NaN,Hacendado,"Aditivos alimentarios, Colorantes alimentarios",Mercadona,100g,100g,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,en:4-ultra-processed-food-and-drink-products,not-applicable
3875,Lomo embuchado,150 g,NaN,Realvalle,"Productos a base de carne, Carnes, Carne de ce...",NaN,100g,100g,203.0,kcal,...,g,1.440,g,855.0,kJ,en:meat-other-than-poultry,"en:fish-meat-eggs,en:meat,en:meat-other-than-p...",4.0,en:4-ultra-processed-food-and-drink-products,e
3876,Pollo al curry ecológico,450g,NaN,Dehesa del milagro,"Comidas preparadas, Ecológica",NaN,100g,100g,143.0,kcal,...,g,0.560,g,597.0,kJ,en:one-dish-meals,"en:composite-foods,en:one-dish-meals",3.0,en:3-processed-foods,b


eliminar columnas que tenga todo nan o null

In [8]:
df = df.dropna( how = "all")

In [9]:
df.columns


Index(['product_name_es', 'quantity', 'serving_size', 'brands', 'categories',
       'stores', 'nutrition_data_per', 'nutrition_data_prepared_per',
       'energy-kcal_value', 'energy-kcal_unit', 'energy-kcal_prepared_value',
       'energy-kcal_prepared_unit', 'fat_value', 'fat_unit',
       'saturated-fat_value', 'saturated-fat_unit', 'carbohydrates_value',
       'carbohydrates_unit', 'sugars_value', 'sugars_unit', 'fiber_value',
       'fiber_unit', 'proteins_value', 'proteins_unit', 'salt_value',
       'salt_unit', 'sodium_value', 'sodium_unit', 'energy_value',
       'energy_unit', 'off:food_groups', 'off:food_groups_tags',
       'off:nova_groups', 'off:nova_groups_tags', 'off:nutriscore_grade'],
      dtype='object')

In [10]:
df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3124 non-null   object 
 2   serving_size                 1015 non-null   object 
 3   brands                       3646 non-null   object 
 4   categories                   3588 non-null   object 
 5   stores                       2221 non-null   object 
 6   nutrition_data_per           3878 non-null   object 
 7   nutrition_data_prepared_per  3878 non-null   object 
 8   energy-kcal_value            2729 non-null   float64
 9   energy-kcal_unit             2732 non-null   object 
 10  energy-kcal_prepared_value   14 non-null     float64
 11  energy-kcal_prepared_unit    15 non-null     object 
 12  fat_value                    2923 non-null   float64
 13  fat_unit          

Eliminar columnas que tengan menos del 30% de sus valores 

In [11]:
#army = army.dropna(axis=1, thresh = int(0.7*len(army)))

df = df.dropna(axis=1, thresh = int(0.7*len(df)))

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3124 non-null   object 
 2   brands                       3646 non-null   object 
 3   categories                   3588 non-null   object 
 4   nutrition_data_per           3878 non-null   object 
 5   nutrition_data_prepared_per  3878 non-null   object 
 6   energy-kcal_value            2729 non-null   float64
 7   energy-kcal_unit             2732 non-null   object 
 8   fat_value                    2923 non-null   float64
 9   fat_unit                     2925 non-null   object 
 10  saturated-fat_value          2842 non-null   float64
 11  saturated-fat_unit           2845 non-null   object 
 12  carbohydrates_value          2917 non-null   float64
 13  carbohydrates_unit

eliminar productos de los que no tenemos los nombres. No nos sirvenº

In [13]:
df = df.dropna(subset="product_name_es")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

In [53]:
lista = df["categories"]
lista.to_csv("list")

- Mirar por que no tenemos valores de cal, azucares etc. P.ej hay aguas. En esos casos los valores tienen que ser 0 y no Nan

In [14]:
filtro = df['product_name_es'].str.contains('agua')
filtro.sum()

np.int64(10)

In [15]:
[col for col in df.columns if "nova" in col.lower()] #cuantas columnas de nova hay en el dataframe

['off:nova_groups_tags']

In [16]:
df["off:nova_groups_tags"] #para ver como es el grupo de nova


0        en:4-ultra-processed-food-and-drink-products
1       en:1-unprocessed-or-minimally-processed-foods
2                                en:3-processed-foods
3        en:4-ultra-processed-food-and-drink-products
4                                en:3-processed-foods
                            ...                      
3873     en:4-ultra-processed-food-and-drink-products
3874     en:4-ultra-processed-food-and-drink-products
3875     en:4-ultra-processed-food-and-drink-products
3876                             en:3-processed-foods
3877                             en:3-processed-foods
Name: off:nova_groups_tags, Length: 3711, dtype: object

In [17]:
df.to_csv("df_util.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

In [19]:
columnas_azucar = ["product_name_es", "categories", "nutrition_data_per", "nutrition_data_prepared_per", "carbohydrates_value", "carbohydrates_unit","sugars_value","sugars_unit","off:nova_groups_tags","off:nutriscore_grade" ]
columnas_grasas = ["product_name_es", "categories", "nutrition_data_per", "nutrition_data_prepared_per", "fat_value", "fat_unit","saturated-fat_value","saturated-fat_unit","off:nova_groups_tags","off:nutriscore_grade" ]
columnas_sal = ["product_name_es", "categories", "nutrition_data_per", "nutrition_data_prepared_per", "salt_value", "salt_unit","off:nova_groups_tags","off:nutriscore_grade"]

In [20]:
df_azucares = df[columnas_azucar]
df_grasas = df[columnas_grasas]
df_sal = df[columnas_sal]


In [21]:
df_azucares.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   categories                   3461 non-null   object 
 2   nutrition_data_per           3711 non-null   object 
 3   nutrition_data_prepared_per  3711 non-null   object 
 4   carbohydrates_value          2806 non-null   float64
 5   carbohydrates_unit           2808 non-null   object 
 6   sugars_value                 2741 non-null   float64
 7   sugars_unit                  2743 non-null   object 
 8   off:nova_groups_tags         3710 non-null   object 
 9   off:nutriscore_grade         3708 non-null   object 
dtypes: float64(2), object(8)
memory usage: 318.9+ KB


vamos a agrupar los valores de la columna categories

In [22]:
df["off:food_groups_tags"].nunique()

43

In [23]:
lista_tags = df["off:food_groups_tags"].unique().tolist()
len(lista_tags)

44

In [24]:
lista_tags

['en:fats-and-sauces,en:dressings-and-sauces',
 'en:milk-and-dairy-products,en:dairy-desserts',
 'en:cereals-and-potatoes,en:breakfast-cereals',
 'en:sugary-snacks,en:biscuits-and-cakes',
 'en:fruits-and-vegetables,en:soups',
 'en:milk-and-dairy-products,en:cheese',
 'en:fish-meat-eggs,en:processed-meat',
 'en:beverages,en:unsweetened-beverages',
 'en:milk-and-dairy-products,en:milk-and-yogurt',
 'en:beverages,en:sweetened-beverages',
 'en:sugary-snacks,en:sweets',
 'en:sugary-snacks,en:chocolate-products',
 'en:fish-meat-eggs,en:meat,en:poultry',
 'en:cereals-and-potatoes,en:bread',
 'en:cereals-and-potatoes,en:cereals',
 'en:fats-and-sauces,en:fats',
 'en:composite-foods,en:one-dish-meals',
 'en:cereals-and-potatoes,en:legumes',
 'en:beverages,en:plant-based-milk-substitutes',
 'en:beverages,en:artificially-sweetened-beverages',
 'en:alcoholic-beverages',
 nan,
 'en:salty-snacks,en:appetizers',
 'en:fruits-and-vegetables,en:vegetables',
 'en:salty-snacks,en:salty-and-fatty-products',

In [25]:
df["grupo_principal"] = df["off:food_groups_tags"].str.split(",").str[0]


In [26]:
mapa_grupos = {
    "en:fats-and-sauces": "Grasas y salsas",
    "en:milk-and-dairy-products": "Lácteos",
    "en:cereals-and-potatoes": "Cereales y tubérculos",
    "en:sugary-snacks": "Dulces y snacks",
    "en:fruits-and-vegetables": "Frutas y verduras",
    "en:fish-meat-eggs": "Proteínas animales",
    "en:beverages": "Bebidas",
    "en:composite-foods": "Comidas preparadas",
    "en:salty-snacks": "Snacks salados",
    "en:alcoholic-beverages": "Bebidas alcohólicas"
}


In [27]:
df["categoria_simplificada"] = df["grupo_principal"].map(mapa_grupos)


In [28]:
df["categoria_simplificada"].unique()


array(['Grasas y salsas', 'Lácteos', 'Cereales y tubérculos',
       'Dulces y snacks', 'Frutas y verduras', 'Proteínas animales',
       'Bebidas', 'Comidas preparadas', 'Bebidas alcohólicas', nan,
       'Snacks salados'], dtype=object)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

In [37]:
df["categoria_simplificada"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
3873    False
3874     True
3875    False
3876    False
3877     True
Name: categoria_simplificada, Length: 3711, dtype: bool

In [39]:
# Filtramos los productos sin categoría simplificada
sin_categoria = df[df["categoria_simplificada"].isna()]

# Vemos cuántos son y qué información útil tienen
print("Productos sin categoría:", len(sin_categoria))
sin_categoria[["product_name_es", "categories", "off:food_groups_tags"]].head(15)


Productos sin categoría: 531


,product_name_es,categories,off:food_groups_tags
100,Bocados marinados,"en:Meat alternatives, Carnes vegetales",NaN
125,Escamas de sal marina natural,"Condimentos, Sales, en:groceries",NaN
129,Pan redondo sin gluten,"en:Alimentos de origen vegetal, en:Alimentos y...",NaN
139,Cappuccino,"Bebidas y preparaciones de bebidas, Alimentos ...",NaN
144,Palomitas sabor mantequilla para microondas,"Lanches comida, Pipocas",NaN
174,White Chocolate,NaN,NaN
197,Colágeno con magnesio,Suplementos dietéticos,NaN
200,Hamburguesa ecológica de vacuno origen Pirineos,"Productos a base de carne, Carne de vacuno y s...",NaN
231,Wieners classic,Productos a base de carne,NaN
240,Crema facial Vitamina C,NaN,NaN


In [40]:
# Cuántos tienen NaN en la categoría simplificada pero sí valor en categories
faltan = df[df["categoria_simplificada"].isna() & df["categories"].notna()]
print(len(faltan))


281


In [42]:
faltan["categories"].unique()

array(['en:Meat alternatives, Carnes vegetales',
       'Condimentos, Sales, en:groceries',
       'en:Alimentos de origen vegetal, en:Alimentos y bebidas de origen vegetal, en:Cereales y patatas, en:Panes, en:Panes de molde',
       'Bebidas y preparaciones de bebidas, Alimentos y bebidas de origen vegetal, Bebidas, Lácteos, Alimentos de origen vegetal, Preparaciones de bebidas, Cafés, Leches evaporadas, Bebidas instantáneas, Cafés solubles, en:Coffee milks, Cappucino en polvo',
       'Lanches comida, Pipocas', 'Suplementos dietéticos',
       'Productos a base de carne, Carne de vacuno y sus productos, Preparados de carne, en:Beef preparations, en:Meat patties, en:Beef patties',
       'Productos a base de carne',
       'Aditivos alimentarios, Levaduras de pastelería, en:Aditivos alimentarios  Levadura en polvo o gasificante',
       'Lácteos, Leche condensada',
       'Cacao y sus productos, Cacaos y chocolates en polvo, Cacaos en polvo',
       'Alimentos e bebidas à base de plan

In [43]:
category_aux_map = {
    # Cárnicos y pescados
    "Carne": "Productos cárnicos",
    "Cárnico": "Productos cárnicos",
    "Pavo": "Productos cárnicos",
    "Jamón": "Productos cárnicos",
    "Pollo": "Productos cárnicos",
    "Embutido": "Productos cárnicos",
    "Morcilla": "Productos cárnicos",
    "Sobrasada": "Productos cárnicos",
    "Croquetas": "Productos cárnicos",
    "Jurel": "Productos cárnicos",
    "Pescad": "Pescado y mariscos",
    "Mariscos": "Pescado y mariscos",
    "Atún": "Pescado y mariscos",
    "Conserva de pescado": "Pescado y mariscos",
    "Marinado": "Pescado y mariscos",

    # Vegetales y frutas
    "Verdura": "Frutas y verduras",
    "Vegetal": "Frutas y verduras",
    "Hortalizas": "Frutas y verduras",
    "Fruta": "Frutas y verduras",
    "Cítricos": "Frutas y verduras",
    "Legumbres": "Frutas y verduras",
    "Germinados": "Frutas y verduras",
    "Judías": "Frutas y verduras",
    "Guisantes": "Frutas y verduras",
    "Quinoa": "Cereales y derivados",

    # Lácteos
    "Lácteos": "Lácteos y derivados",
    "Leche": "Lácteos y derivados",
    "Queso": "Lácteos y derivados",
    "Yogur": "Lácteos y derivados",
    "Nata": "Lácteos y derivados",
    "Mantequilla": "Lácteos y derivados",
    "Helado": "Lácteos y derivados",

    # Sustitutos de lácteos y carne
    "Vegetales": "Alternativas vegetales",
    "Tofu": "Alternativas vegetales",
    "Soja": "Alternativas vegetales",
    "Proteína vegetal": "Alternativas vegetales",
    "Carne vegetal": "Alternativas vegetales",
    "Queso vegetal": "Alternativas vegetales",
    "Leche vegetal": "Alternativas vegetales",
    "Empanados vegetarianos": "Alternativas vegetales",
    "Salchichas vegetales": "Alternativas vegetales",

    # Cereales, panadería y bollería
    "Cereal": "Cereales y derivados",
    "Pan": "Cereales y derivados",
    "Harina": "Cereales y derivados",
    "Bollo": "Bollería y repostería",
    "Galletas": "Bollería y repostería",
    "Bizcocho": "Bollería y repostería",
    "Rosquilletas": "Bollería y repostería",
    "Pastelería": "Bollería y repostería",
    "Repostería": "Bollería y repostería",
    "Cereales para bebés": "Cereales y derivados",

    # Snacks y dulces
    "Chocolate": "Snacks dulces / Cacao y chocolates",
    "Cacao": "Snacks dulces / Cacao y chocolates",
    "Dulce": "Snacks dulces / Cacao y chocolates",
    "Chuches": "Snacks dulces / Cacao y chocolates",
    "Gominolas": "Snacks dulces / Cacao y chocolates",
    "Postres": "Snacks dulces / Cacao y chocolates",
    "Natillas": "Snacks dulces / Cacao y chocolates",
    "Polvo hornear": "Bollería y repostería",

    # Bebidas
    "Bebidas": "Bebidas",
    "Refresco": "Bebidas",
    "Café": "Bebidas calientes",
    "Té": "Bebidas calientes",
    "Infusión": "Bebidas calientes",
    "Cacao en polvo": "Bebidas calientes",
    "Zumos": "Bebidas",
    "Energéticas": "Bebidas",
    "Agua": "Bebidas",
    "Licores": "Bebidas alcohólicas",
    "Alcohol": "Bebidas alcohólicas",
    "Cerveza": "Bebidas alcohólicas",
    "Vino": "Bebidas alcohólicas",

    # Condimentos y grasas
    "Condimentos": "Condimentos y salsas",
    "Salsas": "Condimentos y salsas",
    "Aceite": "Grasas y aceites",
    "Vinagre": "Grasas y aceites",
    "Especias": "Condimentos y salsas",
    "Pimentón": "Condimentos y salsas",
    "Sal": "Condimentos y salsas",
    "Caldos": "Condimentos y salsas",
    "Sazonador": "Condimentos y salsas",
    "Aditivos": "Condimentos y salsas",
    "Colorantes": "Condimentos y salsas",

    # Endulzantes
    "Endulzantes": "Sustitutos del azúcar",
    "Edulcorantes": "Sustitutos del azúcar",
    "Siropes": "Sustitutos del azúcar",
    "Melazas": "Sustitutos del azúcar",
    "Xilitol": "Sustitutos del azúcar",
    "Estevia": "Sustitutos del azúcar",
    "Erythritol": "Sustitutos del azúcar",

    # Suplementos y dietéticos
    "Suplementos": "Suplementos / dietéticos",
    "Proteína": "Suplementos / dietéticos",
    "Culturismo": "Suplementos / dietéticos",
    "Dietético": "Suplementos / dietéticos",
    "Espirulina": "Suplementos / dietéticos",
    "Complemento": "Suplementos / dietéticos",

    # Infantil
    "Infantil": "Alimentación infantil",
    "Bebé": "Alimentación infantil",
    "Papilla": "Alimentación infantil",
    "Leche para bebé": "Alimentación infantil",

    # Otros
    "Comida de perro": "No alimentario",
    "Productos no alimenticios": "No alimentario",
    "Tipo de producto incorrecto": "No alimentario",
    "Decoraciones alimentarias": "Otros",
    "Alimentos festivos": "Otros",
    "Refrigerados": "Otros",
    "Congelados": "Otros"
}


In [44]:
def completar_categoria(row):
    if pd.notna(row["categoria_simplificada"]):
        return row["categoria_simplificada"]
    if isinstance(row["categories"], str):
        for clave, valor in category_aux_map.items():
            if clave.lower() in row["categories"].lower():
                return valor
    return np.nan

df["categoria_simplificada"] = df.apply(completar_categoria, axis=1)


In [45]:
faltantes = df["categoria_simplificada"].isna().sum()
print(f"Faltan {faltantes} categorías por asignar.")


Faltan 277 categorías por asignar.


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3711 entries, 0 to 3877
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   product_name_es              3711 non-null   object 
 1   quantity                     3030 non-null   object 
 2   brands                       3512 non-null   object 
 3   categories                   3461 non-null   object 
 4   nutrition_data_per           3711 non-null   object 
 5   nutrition_data_prepared_per  3711 non-null   object 
 6   energy-kcal_value            2626 non-null   float64
 7   energy-kcal_unit             2629 non-null   object 
 8   fat_value                    2810 non-null   float64
 9   fat_unit                     2812 non-null   object 
 10  saturated-fat_value          2735 non-null   float64
 11  saturated-fat_unit           2738 non-null   object 
 12  carbohydrates_value          2806 non-null   float64
 13  carbohydrates_unit     

In [50]:
resto = df[df["categoria_simplificada"].isna()][["product_name_es", "categories"]]
resto["product_name_es"].unique().tolist()

['Palomitas sabor mantequilla para microondas',
 'White Chocolate',
 'Crema facial Vitamina C',
 'Palomitas dulces para microondas',
 'Cola Cao Original',
 'Piña en su jugo',
 'Mejillones en escabeche',
 'XuFatopía',
 'Garbanzo Lechoso Andaluz',
 'Bocaditos tomate y orégano',
 'Barritas de almendra, chocolate blanco con frutos rojos',
 'Galletas mallorquinas integrales receta tradicional',
 'Leche UHT desnatada',
 'Palomitas de maiz',
 'AVECREM',
 'Tortillas chips',
 'Sal fina con salero',
 'Bimbachitos de Canarias',
 'Caprichos espinacas, pasas y piñones',
 'Cereales rellenos de chocolate y avellanas',
 'Azúcar de coco bio',
 'Bizcochos',
 'Galleta ecológica 240 gr',
 'Tortillitas de camarón',
 'Hojaldrina',
 'Chips fleur de truffe',
 'Mast-Lax',
 'Helios Diet - Mermelada de melocotón extra',
 'Orejones de albaricoque',
 'Be Zero',
 'LOMOS DE BONITO',
 'Pechuga Fileteada de pollo rural',
 'Tortas de aceite de oliva',
 'mediterraneas',
 'Collmar con magnesio',
 'Sirope de chocolate',
 